# Example with Resnet50

In [1]:
# Reference: https://keras.io/api/applications/#resnet50
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.layers import Input, AveragePooling2D, Flatten, Dense, Dropout
import numpy as np

baseModel = ResNet50(weights="imagenet", include_top=True, input_tensor=Input(shape=(224, 224, 3)))

In [2]:
baseModel.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [3]:
def classify_image(img_path, model):
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    preds = model.predict(x)
    return preds # [(class, description, probability)]


In [4]:
img_path = 'dataset-resized/cardboard/cardboard1.jpg'
preds = classify_image(img_path, baseModel)
print('Predicted:', decode_predictions(preds, top=3)[0])

Predicted: [('n02971356', 'carton', 0.5071726), ('n03291819', 'envelope', 0.16329092), ('n02808304', 'bath_towel', 0.084289275)]


In [5]:
img_path = 'dataset-resized/glass/glass1.jpg'
preds = classify_image(img_path, baseModel)
print('Predicted:', decode_predictions(preds, top=3)[0])

Predicted: [('n02841315', 'binoculars', 0.80819815), ('n03692522', 'loupe', 0.0636286), ('n04009552', 'projector', 0.030073166)]


In [6]:
# Reference: https://www.pyimagesearch.com/2020/04/27/fine-tuning-resnet-with-keras-tensorflow-and-deep-learning/
baseModel = ResNet50(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(256, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(6, activation="softmax")(headModel)
model = tf.keras.Model(inputs=baseModel.input, outputs=headModel)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [7]:
# loop over all layers in the base model and freeze them so they will not be updated during the training process
for layer in baseModel.layers:\
    layer.trainable = False

In [8]:
# compile the model
model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

In [ ]:
# TODO: create train/val/test datasets

In [ ]:
# TODO: train the model
#history = model.fit_generator(train_data, validation_data=val_data)

In [ ]:
# TODO: test 
# pred = model.predict_generator(test_data)

In [ ]:
# TODO: find class for each pred

In [ ]:
#TODO: save model
# model.save('models/resnet50', save_format="h5")

In [ ]:
# TODO: plot results